# Gradient Boosting Tree

In [2]:
# librerías auxiliares
import numpy as np
import pandas as pd
from itertools import *

# modulos de sklearn
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.class_weight import compute_class_weight
from imblearn.under_sampling import RandomUnderSampler, NearMiss

# metricas de desempeño
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

Importamos los datasets para cada modelo

In [3]:
data_estimador1 = pd.read_csv("dataset_clasificador_binario.csv")
data_estimador2 = pd.read_csv("dataset_clasificador_multiclase.csv")

## Construcción del modelo para identificar si habrá un fallo en las maquinas

El dataset que se usará para la construcción del modelo es el siguiente:

In [4]:
data_estimador1

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,H,L,M
0,-0.951417,-0.946356,0.067484,0.283054,-1.695647,0,0.0,0.0,1.0
1,-0.901428,-0.878954,-0.729604,0.634238,-1.648511,0,0.0,1.0,0.0
2,-0.951417,-1.013759,-0.227940,0.945286,-1.617087,0,0.0,1.0,0.0
3,-0.901428,-0.946356,-0.590253,-0.048061,-1.585664,0,0.0,1.0,0.0
4,-0.901428,-0.878954,-0.729604,0.002108,-1.554240,0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
9968,-0.601495,-1.081162,0.362908,-1.051442,-1.475680,0,0.0,0.0,1.0
9969,-0.551506,-1.081162,0.518981,-0.820665,-1.428545,0,1.0,0.0,0.0
9970,-0.501517,-0.946356,0.591443,-0.660124,-1.349985,0,0.0,0.0,1.0
9971,-0.501517,-0.878954,-0.729604,0.854982,-1.302849,0,1.0,0.0,0.0


In [20]:
def binary_gbt(x, y, num_trees, min_sample_split):
    """funcion que realiza experimentos de arboles de decision
    X: matriz con las caractersiticas
    Y: matriz de numpy con etiquetas
    num_trees: list[int] lista con el número de arboles usado para el RF
    min_sample_split: list[int] lista con el número mínimo de muestras requeridas para dividir un nodo interno
    max_feature: list[int] lista con el número de características a considerar al buscar la mejor división  
    
    retorna: dataframe con:
        retorna: pd.Dataframe con las siguientes columnas:
        - 3 columnas con los tres parametros: numero de arboles, muestras minimas para la particion de nodos, maximo numero de caracteristicas
        - recall en el cojunto entrenamiento (promedio de los 5 folds)
        - recall en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del recall promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - auc roc en el cojunto entrenamiento (promedio de los 5 folds)
        - auc roc en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del auc roc promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - f1 score en el cojunto entrenamiento (promedio de los 5 folds)
        - f1 score en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del f1 score promedio en el cojunto test 
            (desviacion estandar de los 5 folds)
    """
    skf = StratifiedKFold(n_splits=5)
    # crear una lista con la combinaciones de los elementos de cada list
    gbt_params = list(product(num_trees, min_sample_split))
    resultados = pd.DataFrame()
    idx = 0
    for params in gbt_params:
        tree, min_split = params
        print("parametros usados", params) # puede usar para ver los params
        errores_train_recall = []
        errores_test_recall = []

        errores_train_auc = []
        errores_test_auc = []

        for train_index, test_index in skf.split(x, y):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index] 

            # calculamos los pesos para cada clase
            class_weight = compute_class_weight(class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train )
            
            # Balancear los datos
            nm = RandomUnderSampler(sampling_strategy=0.5)
            X_train, y_train = nm.fit_resample(X_train, y_train)

            # definir el modelo
            gbt=GradientBoostingClassifier(n_estimators=tree, min_samples_split=min_split)

            # Establecemos los pesos
            weight = []
            for i in y_train:
              weight.append(class_weight[i])

            # Entrenar el modelo
            gbt.fit(X_train, y_train, weight)

            # prediccion del modelo
            y_train_pred = gbt.predict(X=X_train)
            y_test_pred = gbt.predict(X=X_test)

            #Evaluamos las predicciones del modelo con los datos de train y de test
            # calculo del recall
            errores_train_recall.append(recall_score(y_true = y_train, y_pred = y_train_pred))
            errores_test_recall.append(recall_score(y_true = y_test, y_pred = y_test_pred))
            # calculo del area bajo la curva roc
            errores_train_auc.append(roc_auc_score(y_true=y_train, y_score=gbt.predict_proba(X_train)[:, 1]))
            errores_test_auc.append(roc_auc_score(y_true=y_test, y_score=gbt.predict_proba(X_test)[:, 1]))

        resultados.loc[idx,'número de arboles'] = tree
        resultados.loc[idx,'minimo numero de muestras para split'] = min_split

        resultados.loc[idx,'error de entrenamiento (recall)'] = np.mean(errores_train_recall)
        resultados.loc[idx,'error de prueba (recall)'] = np.mean(errores_test_recall)
        resultados.loc[idx,'error de prueba (intervalo de confianza - recall)'] = np.std(errores_test_recall)

        resultados.loc[idx,'error de entrenamiento (auc)'] = np.mean(errores_train_auc)
        resultados.loc[idx,'error de prueba (auc)'] = np.mean(errores_test_auc)
        resultados.loc[idx,'error de prueba (intervalo de confianza - auc)'] = np.std(errores_test_auc)
        idx= idx +1
        
    return (resultados)

Realizamos los experimentos para encontrar la mejor combinación de hiperparametros

In [21]:
# vamos a realizar los experimentos
x_estimador1 = data_estimador1.drop(['Target'], axis=1).values
y_estimador1 = data_estimador1['Target'].values
resultadosGBT = binary_gbt(x = x_estimador1,y=y_estimador1,
                                 num_trees=[5,10,20,50,100, 150],
                                 min_sample_split = [3,4,5])

parametros usados (5, 3)
parametros usados (5, 4)
parametros usados (5, 5)
parametros usados (10, 3)
parametros usados (10, 4)
parametros usados (10, 5)
parametros usados (20, 3)
parametros usados (20, 4)
parametros usados (20, 5)
parametros usados (50, 3)
parametros usados (50, 4)
parametros usados (50, 5)
parametros usados (100, 3)
parametros usados (100, 4)
parametros usados (100, 5)
parametros usados (150, 3)
parametros usados (150, 4)
parametros usados (150, 5)


ver los 5 mejores modelos

In [22]:
resultadosGBT.sort_values('error de prueba (recall)', ascending=False).head(5)

,número de arboles,minimo numero de muestras para split,error de entrenamiento (recall),error de prueba (recall),error de prueba (intervalo de confianza - recall),error de entrenamiento (auc),error de prueba (auc),error de prueba (intervalo de confianza - auc)
1,5.0,4.0,1.0,0.981818,0.036364,0.951877,0.865113,0.045813
4,10.0,4.0,1.0,0.969697,0.041770,0.955466,0.889296,0.057749
2,5.0,5.0,1.0,0.966667,0.059382,0.948320,0.857347,0.078144
3,10.0,3.0,1.0,0.960606,0.058760,0.957060,0.869176,0.068468
0,5.0,3.0,1.0,0.954545,0.090909,0.952268,0.852515,0.109342


## Construcción del modelo para identificar el tipo de fallo

El dataset que se usará para la construcción del modelo es el siguiente:

In [23]:
data_estimador2

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,H,L,M
0,-0.551506,-0.609342,7.369473,-3.549861,0.551157,2.0,0.0,1.0,0.0
1,-0.551506,-0.676745,-0.718456,2.580797,1.305329,2.0,0.0,1.0,0.0
2,-0.601495,-0.744148,-0.467624,0.132547,1.572432,3.0,0.0,1.0,0.0
3,-0.801450,-1.215968,-1.431932,2.079106,1.698127,1.0,0.0,1.0,0.0
4,-0.851439,-1.283370,-0.707308,1.236266,1.729551,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
325,-0.701473,-0.137523,4.080791,-2.385939,1.729551,3.0,0.0,1.0,0.0
326,-0.751461,-0.339731,-1.365044,2.681135,-1.507104,2.0,0.0,1.0,0.0
327,-0.751461,-0.407134,-0.997157,2.099174,1.242481,1.0,0.0,1.0,0.0
328,-0.851439,-0.474537,-1.125360,1.617551,1.541008,1.0,0.0,1.0,0.0


In [27]:
def multiclass_gbt(x, y, num_trees, min_sample_split, estrategia = 'ovr'):
    """funcion que realiza experimentos de arboles de decision
    X: matriz con las caractersiticas
    Y: matriz de numpy con etiquetas
    num_trees: list[int] lista con el número de arboles usado para el RF
    min_sample_split: list[int] lista con el número mínimo de muestras requeridas para dividir un nodo interno
    max_feature: list[int] lista con el número de características a considerar al buscar la mejor división  
    estrategia: str, valor configurado por defecto para indicar que se usará ovr 
        (para one vs rest)

    retorna: dataframe con:
        retorna: pd.Dataframe con las siguientes columnas:
        - 3 columnas con los tres parametros: numero de arboles, muestras minimas para la particion de nodos, maximo numero de caracteristicas
        - recall en el cojunto entrenamiento (promedio de los 5 folds)
        - recall en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del recall promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - auc roc en el cojunto entrenamiento (promedio de los 5 folds)
        - auc roc en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del auc roc promedio en el cojunto test 
            (desviacion estandar de los 5 folds)

        - f1 score en el cojunto entrenamiento (promedio de los 5 folds)
        - f1 score en el cojunto test (promedio de los 5 folds)
        - intervalo de confianza del f1 score promedio en el cojunto test 
            (desviacion estandar de los 5 folds)
    """
    skf = StratifiedKFold(n_splits=5)
    # crear una lista con la combinaciones de los elementos de cada list
    gbt_params = list(product(num_trees, min_sample_split))
    resultados = pd.DataFrame()
    idx = 0
    for params in gbt_params:
        tree, min_split = params
        print("parametros usados", params) # puede usar para ver los params
        errores_train_recall = []
        errores_test_recall = []

        errores_train_auc = []
        errores_test_auc = []

        for train_index, test_index in skf.split(x, y):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]  
            
            # Balancear los datos
            smote = SMOTE(k_neighbors=3) 
            X_train, y_train = smote.fit_resample(X_train, y_train)

            # definir el modelo
            gbt=GradientBoostingClassifier(n_estimators=tree, min_samples_split=min_split)

            # se usará la estrategia ovr
            gbt = OneVsRestClassifier(gbt)  

            gbt.fit(X_train, y_train)

            # prediccion del modelo
            y_train_pred = gbt.predict(X=X_train)
            y_test_pred = gbt.predict(X=X_test)

            #Evaluamos las predicciones del modelo con los datos de train y de test
            # calculo del recall
            errores_train_recall.append(recall_score(y_true = y_train, y_pred = y_train_pred, average='weighted'))
            errores_test_recall.append(recall_score(y_true = y_test, y_pred = y_test_pred, average='weighted'))
            # calculo del area bajo la curva roc
            errores_train_auc.append(roc_auc_score(y_true=y_train, y_score=gbt.predict_proba(X_train), multi_class='ovr'))
            errores_test_auc.append(roc_auc_score(y_true=y_test, y_score=gbt.predict_proba(X_test), multi_class='ovr'))

        resultados.loc[idx,'número de arboles'] = tree
        resultados.loc[idx,'minimo numero de muestras para split'] = min_split
        resultados.loc[idx,'estrategia'] = estrategia

        resultados.loc[idx,'error de entrenamiento (recall)'] = np.mean(errores_train_recall)
        resultados.loc[idx,'error de prueba (recall)'] = np.mean(errores_test_recall)
        resultados.loc[idx,'error de prueba (intervalo de confianza - recall)'] = np.std(errores_test_recall)

        resultados.loc[idx,'error de entrenamiento (auc)'] = np.mean(errores_train_auc)
        resultados.loc[idx,'error de prueba (auc)'] = np.mean(errores_test_auc)
        resultados.loc[idx,'error de prueba (intervalo de confianza - auc)'] = np.std(errores_test_auc)

        idx= idx +1
        
    return (resultados)

Realizamos los experimentos para encontrar la mejor combinación de hiperparametros

In [28]:
# vamos a realizar los experimentos
x_estimador2 = data_estimador2.drop(['Failure Type'], axis=1).values
y_estimador2 = data_estimador2['Failure Type'].values
resultadosGBT_ovr = multiclass_gbt(x = x_estimador2,y=y_estimador2,
                                 num_trees=[5,10,20,50,100, 150],
                                 min_sample_split = [3,4,5])

parametros usados (5, 3)
parametros usados (5, 4)
parametros usados (5, 5)
parametros usados (10, 3)
parametros usados (10, 4)
parametros usados (10, 5)
parametros usados (20, 3)
parametros usados (20, 4)
parametros usados (20, 5)
parametros usados (50, 3)
parametros usados (50, 4)
parametros usados (50, 5)
parametros usados (100, 3)
parametros usados (100, 4)
parametros usados (100, 5)
parametros usados (150, 3)
parametros usados (150, 4)
parametros usados (150, 5)


ver los 5 mejores modelos

In [29]:
resultadosGBT_ovr.sort_values('error de prueba (auc)', ascending=False).head(5)

,número de arboles,minimo numero de muestras para split,estrategia,error de entrenamiento (recall),error de prueba (recall),error de prueba (intervalo de confianza - recall),error de entrenamiento (auc),error de prueba (auc),error de prueba (intervalo de confianza - auc)
10,50.0,4.0,ovr,1.0,0.872727,0.072093,1.0,0.985796,0.015088
15,150.0,3.0,ovr,1.0,0.896970,0.068702,1.0,0.984558,0.018631
12,100.0,3.0,ovr,1.0,0.878788,0.072347,1.0,0.984017,0.018068
13,100.0,4.0,ovr,1.0,0.884848,0.093695,1.0,0.983845,0.018950
17,150.0,5.0,ovr,1.0,0.896970,0.084848,1.0,0.983064,0.019562
